In [4]:
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt

In [5]:
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from os import listdir

In [6]:
damagepath = "/code/Project2/COE379L-Project2/damage"
nodamagepath = "/code/Project2/COE379L-Project2/no_damage"

X = []
y = []

# Load damaged images and convert to black and white
for img_name in os.listdir(damagepath):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(damagepath, img_name)
        img = load_img(img_path)
        img_gray = img.convert('L')
        img_array = img_to_array(img_gray)
        X.append(img_array)
        y.append(1)

# Load non-damaged images
for img_name in os.listdir(nodamagepath):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(nodamagepath, img_name)
        img = load_img(img_path)
        img_gray = img.convert('L')
        img_array = img_to_array(img_gray)
        X.append(img_array)
        y.append(0) 

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

print("Loaded images:", X.shape)
print("Labels:", y.shape)

Loaded images: (21322, 128, 128, 1)
Labels: (21322,)


In [7]:
print("Total images:", X.shape)
print("Total labels:", y.shape)

Total images: (21322, 128, 128, 1)
Total labels: (21322,)


In [8]:
print("Image shape (Height x Width x Channels):", X[0].shape)
print("All image shapes unique:", np.unique([img.shape for img in X], axis=0))

Image shape (Height x Width x Channels): (128, 128, 1)
All image shapes unique: [[128 128   1]]


In [9]:
print("Min pixel value:", X.min())
print("Max pixel value:", X.max())

Min pixel value: 0.0
Max pixel value: 255.0


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 43, stratify = y)

In [11]:
# Now I will scale the image to a 1D array and Normalize the Data
image_vector_size = 128 * 128
X_train = X_train.reshape(X_train.shape[0], image_vector_size)
X_test = X_test.reshape(X_test.shape[0], image_vector_size)
X_train_normalized = X_train / 255.0
X_test_normalized = X_test / 255.0


In [12]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
# Here we map all the inputs to every perceptron (1024) creating a dense layer while also having 128*128 perceptrons in the first layer
model.add(Dense(256, input_dim=image_vector_size, activation='relu'))

# Here we create another dense layer except with less perceptrons
model.add(Dense(128, activation='relu'))

# Here we create another dense layer except with less perceptrons
model.add(Dense(64, activation='relu'))

# This is the output layer. The reason it only has 1 perceptron is because we are considering 0 to be 
# a non-destructed building and 1 to be a destructed building
model.add(Dense(1, activation='sigmoid'))

In [13]:
from keras.callbacks import EarlyStopping
# Creating an early stopping condition for the model
# This helps optimize the model
# In this case we look at val_loss (monitor)
# min_delta tells you the improvement required to not stop (.5 = if the epoch results in val_loss improving by < .5, then flag)
# patience tells us how long to continue doing epochs in a row if the flag is raised (flag not raising x times in a row). it then quits
# start_from_epoch tells us when to consider the early stopping
# restore_best_weights tells us to go to the best epoch when done
early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=0, start_from_epoch=5, restore_best_weights=True, verbose=1)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_normalized, y_train, validation_split=0.2, epochs=30, batch_size=4, verbose=2, callbacks=[early_stopping])

Epoch 1/30
3412/3412 - 127s - loss: 0.6668 - accuracy: 0.6544 - val_loss: 0.6461 - val_accuracy: 0.6527 - 127s/epoch - 37ms/step
Epoch 2/30
3412/3412 - 123s - loss: 0.6374 - accuracy: 0.6676 - val_loss: 0.6458 - val_accuracy: 0.6527 - 123s/epoch - 36ms/step
Epoch 3/30
3412/3412 - 122s - loss: 0.6367 - accuracy: 0.6676 - val_loss: 0.6459 - val_accuracy: 0.6527 - 122s/epoch - 36ms/step
Epoch 4/30
3412/3412 - 122s - loss: 0.6367 - accuracy: 0.6676 - val_loss: 0.6458 - val_accuracy: 0.6527 - 122s/epoch - 36ms/step
Epoch 5/30
3412/3412 - 124s - loss: 0.6366 - accuracy: 0.6676 - val_loss: 0.6458 - val_accuracy: 0.6527 - 124s/epoch - 36ms/step
Epoch 6/30
3412/3412 - 123s - loss: 0.6364 - accuracy: 0.6676 - val_loss: 0.6487 - val_accuracy: 0.6527 - 123s/epoch - 36ms/step
Epoch 7/30
3412/3412 - 123s - loss: 0.6362 - accuracy: 0.6676 - val_loss: 0.6462 - val_accuracy: 0.6527 - 123s/epoch - 36ms/step
Epoch 8/30
Restoring model weights from the end of the best epoch: 7.
3412/3412 - 122s - loss: 0.

In [14]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 0.6404153108596802
Test Accuracy: 0.6642438173294067


In [28]:
# CNN LeNet-5
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.optimizers import Adam

# For the CNN reshape the inputs as it will be flattened in the ANN
X_train_cnn = X_train_normalized.reshape(-1, 128, 128, 1)
X_test_cnn = X_test_normalized.reshape(-1, 128, 128, 1)

model_cnn = Sequential()
# Convolutional layer with 6 filters of size 5x5
model_cnn.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(128, 128, 1)))
model_cnn.add(AveragePooling2D(pool_size=(2, 2), padding = 'same'))

# Convolutional layer with 16 filters of size 5x5
model_cnn.add(Conv2D(16, kernel_size=(5, 5), activation='relu',))
model_cnn.add(AveragePooling2D(pool_size=(2, 2), padding = 'same'))

# Flatten the feature maps to feed into the ANN
model_cnn.add(Flatten())

model_cnn.add(Dense(120, activation='relu'))

model_cnn.add(Dense(84, activation='relu'))

# Fully connected layer with num_classes neurons 
model_cnn.add(Dense(1, activation='sigmoid'))

In [29]:
from keras.callbacks import EarlyStopping
# Creating an early stopping condition for the model
# This helps optimize the model
# In this case we look at val_loss (monitor)
# min_delta tells you the improvement required to not stop (.5 = if the epoch results in val_loss improving by < .5, then flag)
# patience tells us how long to continue doing epochs in a row if the flag is raised (flag not raising x times in a row). it then quits
# start_from_epoch tells us when to consider the early stopping
# restore_best_weights tells us to go to the best epoch when done
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_cnn.fit(X_train_cnn, y_train, validation_split=0.2, epochs=15, batch_size=120, verbose=2)

Epoch 1/15
114/114 - 37s - loss: 0.4837 - accuracy: 0.7794 - val_loss: 0.3680 - val_accuracy: 0.8499 - 37s/epoch - 323ms/step
Epoch 2/15
114/114 - 38s - loss: 0.3292 - accuracy: 0.8707 - val_loss: 0.2834 - val_accuracy: 0.8863 - 38s/epoch - 331ms/step
Epoch 3/15
114/114 - 38s - loss: 0.2465 - accuracy: 0.9038 - val_loss: 0.2091 - val_accuracy: 0.9174 - 38s/epoch - 330ms/step
Epoch 4/15
114/114 - 37s - loss: 0.2078 - accuracy: 0.9200 - val_loss: 0.1816 - val_accuracy: 0.9232 - 37s/epoch - 328ms/step
Epoch 5/15
114/114 - 37s - loss: 0.1834 - accuracy: 0.9315 - val_loss: 0.1799 - val_accuracy: 0.9288 - 37s/epoch - 328ms/step
Epoch 6/15
114/114 - 37s - loss: 0.1665 - accuracy: 0.9340 - val_loss: 0.1758 - val_accuracy: 0.9270 - 37s/epoch - 325ms/step
Epoch 7/15
114/114 - 39s - loss: 0.1621 - accuracy: 0.9369 - val_loss: 0.1768 - val_accuracy: 0.9276 - 39s/epoch - 340ms/step
Epoch 8/15
114/114 - 37s - loss: 0.1364 - accuracy: 0.9477 - val_loss: 0.2163 - val_accuracy: 0.9153 - 37s/epoch - 327

In [31]:
# Fit to the test data and analyze accuracy
test_loss, test_accuracy = model_cnn.evaluate(X_test_cnn, y_test, verbose = 0)
print(f"Accuracy on the Test: {test_accuracy}")
print(f"Test loss: {test_loss}")

Accuracy on the Test: 0.9252051711082458
Test loss: 0.2637929320335388
